In [41]:
import pandas as pd
import requests as req
import numpy as np
from tqdm.notebook import tqdm


from bs4 import BeautifulSoup as bs

from joblib import Parallel, delayed
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
url = 'https://chargefinder.com/en/washington/charging-station/m76w89'

html = req.get(url).content 

soup = bs(html, 'html.parser')

In [6]:
soup.find_all('div')

[<div id="app"></div>]

In [36]:
df = pd.read_csv('data/Electric_Vehicle_Charging_Stations.csv')

In [37]:
df

,Station Name,Street Address,City,Access Days Time,EV Level1 EVSE Num,EV Level2 EVSE Num,EV DC Fast Count,EV Other Info,New Georeferenced Column
0,BMW OF DARIEN,138-142 Ledge Rd,Darien,24 hours daily,NONE,2,NONE,NONE,POINT (-73.4764687 41.072882)
1,Dunkin’ - Tesla Supercharger,893 E Main St,Meriden,24 hours daily; for Tesla use only,NONE,NONE,8,NONE,POINT (-72.773473 41.527367)
2,Town of Beacon Falls - Commuter Lot,105 N Main St,Beacon Falls,24 hours daily,NONE,1,NONE,NONE,POINT (-73.065583 41.44548100000001)
3,OLD SAYBROOK VW,319 Middlesex Turnpike,Old Saybrook,24 hours daily,NONE,2,NONE,NONE,POINT (-72.3825 41.3102778)
4,Fairfield Rail Station,80 Mill Plain Rd,Fairfield,24 hours daily,NONE,2,NONE,NONE,POINT (-73.264511 41.143125)
...,...,...,...,...,...,...,...,...,...
380,Lee Partyka Chevrolet Mazda Isuzu,200 Skiff St,Hamden,Dealership business hours,NONE,3,NONE,NONE,POINT (-72.9100121 41.3729709)
381,777 MAIN ST,71 Asylum St,Hartford,24 hours daily,NONE,10,1,NONE,POINT (-72.674468 41.766676)
382,Norwich Public Utilities,173 N Main St,Norwich,24 hours daily,NONE,4,NONE,NONE,POINT (-72.063516 41.52757900000001)
383,Fairfield Plaza,116 Danbury Rd,New Milford,24 hours daily,NONE,NONE,1,NONE,POINT (-73.41763 41.55735)


In [38]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 385 entries, 0 to 384
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Station Name              385 non-null    object
 1   Street Address            385 non-null    object
 2   City                      385 non-null    object
 3   Access Days Time          385 non-null    object
 4   EV Level1 EVSE Num        385 non-null    object
 5   EV Level2 EVSE Num        385 non-null    object
 6   EV DC Fast Count          385 non-null    object
 7   EV Other Info             385 non-null    object
 8   New Georeferenced Column  385 non-null    object
dtypes: object(9)
memory usage: 237.7 KB


In [44]:
df1 = df.replace('NONE',np.nan)

In [45]:
df1.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 385 entries, 0 to 384
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Station Name              385 non-null    object
 1   Street Address            385 non-null    object
 2   City                      385 non-null    object
 3   Access Days Time          385 non-null    object
 4   EV Level1 EVSE Num        9 non-null      object
 5   EV Level2 EVSE Num        341 non-null    object
 6   EV DC Fast Count          54 non-null     object
 7   EV Other Info             2 non-null      object
 8   New Georeferenced Column  385 non-null    object
dtypes: object(9)
memory usage: 205.6 KB


In [46]:
df1.drop(columns= 'EV Level1 EVSE Num',inplace=True)
df1.drop(columns= 'EV DC Fast Count',inplace=True)
df1.drop(columns= 'EV Other Info',inplace=True)

In [50]:
df1 = df1.dropna()

In [51]:
df1.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 341 entries, 0 to 384
Data columns (total 6 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Station Name              341 non-null    object
 1   Street Address            341 non-null    object
 2   City                      341 non-null    object
 3   Access Days Time          341 non-null    object
 4   EV Level2 EVSE Num        341 non-null    object
 5   New Georeferenced Column  341 non-null    object
dtypes: object(6)
memory usage: 152.1 KB


In [53]:
df1 = df1.drop_duplicates()

In [54]:
df1.shape

(341, 6)

In [55]:
df1.City.unique()

array(['Darien', 'Beacon Falls', 'Old Saybrook', 'Fairfield', 'Norwich',
       'Glastonbury', 'Groton', 'Waterbury', 'Farmington', 'Shelton',
       'Wallingford', 'Rocky Hill', 'New Haven', 'Easton', 'Colebrook',
       'Mystic', 'Windsor Locks', 'Storrs', 'Madison', 'New Britain',
       'Mansfield', 'Torrington', 'Harwinton', 'Middletown',
       'North Windham', 'Westbrook', 'Bristol', 'Branford', 'Manchester',
       'Meriden', 'Guilford', 'Simsbury', 'Hartford', 'Stratford',
       'Stonington', 'Westport', 'Stamford', 'Niantic', 'New Milford',
       'Newtown', 'Bloomfield', 'Washington', 'North Haven',
       'Stafford Springs', 'West Hartford', 'Newington', 'Norwalk',
       'Southbury', 'Ridgefield', 'Mashantucket', 'East Hartford',
       'Ledyard', 'Plainville', 'Greenwich', 'New London', 'Windham',
       'Danbury', 'Canton', 'Durham', 'Southington', 'New Fairfield',
       'Orange', 'West Haven', 'Bridgeport', 'Coventry', 'Uncasville',
       'Enfield', 'Ellington', 'Ber

In [56]:
df1[['Latitude', 'Longitude']] = df1['New Georeferenced Column'].str.strip('POINT ()').str.split(expand=True)

In [58]:
df1.drop(columns='New Georeferenced Column',inplace= True )

In [61]:
df1['Access Days Time'].unique()

array(['24 hours daily',
       '24 hours daily; guest use only; see front desk for access',
       '6am-12am daily', 'Dealership business hours',
       '24 hours daily; for customer use only; see front desk for access',
       '24 hours daily; pay lot', '24 hours daily; for customer use only',
       '8am-3pm M-F', '24 hours daily; permit lot', '6am-12pm daily',
       '7am-7pm M-F', 'Garage business hours',
       'MO: Not Specified; TU: Not Specified; WE: Not Specified; TH: Not Specified; FR: Not Specified; SA: Not Specified; SU: Not Specified',
       'Restricted to garage hours; pay lot',
       'MO: 12:00am-12:00am; TU: 12:00am-12:00am; WE: 12:00am-12:00am; TH: 12:00am-12:00am; FR: 12:00am-12:00am; SA: 12:00am-12:00am; SU: 12:00am-12:00am',
       'School hours', '6am-12am',
       '8am-10pm daily; for customer use only; see inside for access',
       '24 hours daily; for member use only; see valet for access',
       '24 hours daily; overnight valet fee',
       '24 hours daily

In [64]:
def clean(x):
    x=str(x)
    if '24' in x:
        return '24 Hours'
    else:
        return 'Regular'
df1['Access Days Time'] = df1['Access Days Time'].apply(clean)

# 

In [65]:
df1

,Station Name,Street Address,City,Access Days Time,EV Level2 EVSE Num,Latitude,Longitude
0,BMW OF DARIEN,138-142 Ledge Rd,Darien,24 Hours,2,-73.4764687,41.072882
2,Town of Beacon Falls - Commuter Lot,105 N Main St,Beacon Falls,24 Hours,1,-73.065583,41.44548100000001
3,OLD SAYBROOK VW,319 Middlesex Turnpike,Old Saybrook,24 Hours,2,-72.3825,41.3102778
4,Fairfield Rail Station,80 Mill Plain Rd,Fairfield,24 Hours,2,-73.264511,41.143125
5,FOUNDRY66,50 Franklin St,Norwich,24 Hours,2,-72.0741188,41.525611
...,...,...,...,...,...,...,...
379,Yankee Doodle,10 Burnell Boulevard,Norwalk,24 Hours,1,-73.414932,41.11803
380,Lee Partyka Chevrolet Mazda Isuzu,200 Skiff St,Hamden,Regular,3,-72.9100121,41.3729709
381,777 MAIN ST,71 Asylum St,Hartford,24 Hours,10,-72.674468,41.766676
382,Norwich Public Utilities,173 N Main St,Norwich,24 Hours,4,-72.063516,41.52757900000001


In [68]:
df1 = df1.rename(columns= {'EV Level2 EVSE Num': 'EVSE units'})

In [69]:
df1

,Station Name,Street Address,City,Access Days Time,EVSE units,Latitude,Longitude
0,BMW OF DARIEN,138-142 Ledge Rd,Darien,24 Hours,2,-73.4764687,41.072882
2,Town of Beacon Falls - Commuter Lot,105 N Main St,Beacon Falls,24 Hours,1,-73.065583,41.44548100000001
3,OLD SAYBROOK VW,319 Middlesex Turnpike,Old Saybrook,24 Hours,2,-72.3825,41.3102778
4,Fairfield Rail Station,80 Mill Plain Rd,Fairfield,24 Hours,2,-73.264511,41.143125
5,FOUNDRY66,50 Franklin St,Norwich,24 Hours,2,-72.0741188,41.525611
...,...,...,...,...,...,...,...
379,Yankee Doodle,10 Burnell Boulevard,Norwalk,24 Hours,1,-73.414932,41.11803
380,Lee Partyka Chevrolet Mazda Isuzu,200 Skiff St,Hamden,Regular,3,-72.9100121,41.3729709
381,777 MAIN ST,71 Asylum St,Hartford,24 Hours,10,-72.674468,41.766676
382,Norwich Public Utilities,173 N Main St,Norwich,24 Hours,4,-72.063516,41.52757900000001
